# Laboratório: Produção de Dados com Kafka e MinIO

In [ ]:
!pip install kafka-python minio pandas

## 1. Importação das bibliotecas necessárias

In [ ]:
import random
import json
import pandas as pd
import time
from kafka import KafkaProducer
from minio import Minio
from io import BytesIO

## 2. Configurar cliente MinIO

In [ ]:
minio_client = Minio(
    "minio:9000",
    access_key="4PRJYFLGzQYTnOJGH1gA",
    secret_key="ovBkCsqh2cXNkyoteCzQMV5JWCUk5tHfsG1GwYbD",
    secure=False
)

## 3. Carregar dados de clientes

In [ ]:
# Listar e ler todos os arquivos Parquet de clientes armazenados no bucket 'raw' do MinIO.
customer_objects = minio_client.list_objects("raw", prefix="pagila_db/customer/", recursive=True)
customer_list = []
# Para cada objeto encontrado, ler o conteúdo e adicionar ao DataFrame.
for obj in customer_objects:
    data = minio_client.get_object("raw", obj.object_name)
    df = pd.read_parquet(BytesIO(data.read()))
    customer_list.append(df)
# Concatenar todos os DataFrames em um único DataFrame de clientes.
customers = pd.concat(customer_list, ignore_index=True)
# Criar coluna 'name' unindo primeiro e último nome.
customers['name'] = customers['first_name'] + ' ' + customers['last_name']
# Visualizar os 10 primeiros nomes de clientes.
customers['name'].head(10)


## 4. Carregar dados de filmes

In [ ]:
# Listar e ler todos os arquivos Parquet de filmes armazenados no bucket 'raw' do MinIO.
film_objects = minio_client.list_objects("raw", prefix="pagila_db/film/", recursive=True)
films_list = []
# Para cada objeto encontrado, ler o conteúdo e adicionar ao DataFrame.
for obj in film_objects:
    data = minio_client.get_object("raw", obj.object_name)
    df = pd.read_parquet(BytesIO(data.read()))
    films_list.append(df)
# Concatenar todos os DataFrames em um único DataFrame de filmes.
films = pd.concat(films_list, ignore_index=True)
# Visualizar os 10 primeiros títulos de filmes.
films['title'].head(10)

## 5 Montar uma lista de reviews fictícias

In [ ]:
# Listas de reviews fictícias separadas por sentimento: positivo, neutro e negativo.
positive_review = [
    "Um filme incrível! Recomendo muito.",
    "Adorei a atuação e o roteiro.",
    "Excelente produção, vale a pena assistir.",
    "Uma experiência cinematográfica fantástica.",
    "Muito divertido e envolvente!"
 ]
# Reviews neutras.
neutral_review = [
    "O filme é bom, mas nada de especial.",
    "Entretenimento razoável.",
    "Achei interessante, mas poderia ser melhor.",
    "Um filme mediano.",
    "Cumpre o que promete."
 ]
# Reviews negativas.
negative_review = [
    "Não gostei muito do filme.",
    "Achei a história fraca.",
    "Esperava mais dos atores.",
    "O filme é cansativo.",
    "Não recomendo."
 ]

## 6. Função para gerar uma review aleatória

In [ ]:
# Dicionário para armazenar a review gerada.
review = {
    'text': None,
    'rating': None
}

# Função para gerar uma review aleatória com base em sentimentos e atribuir uma nota.
def generate_review():
    tipo = random.choices(
        ['positiva', 'neutra', 'negativa'],
        weights=[0.3, 0.5, 0.2] # Probabilidades para cada tipo de sentimento
    )[0]
    if tipo == 'positiva':
        review['text'] = random.choice(positive_review)
        review['rating'] = random.randint(7, 10) # Notas mais altas para reviews positivas
        return review
    elif tipo == 'neutra':
        review['text'] = random.choice(neutral_review)
        review['rating'] = random.randint(5, 6) # Notas intermediárias para reviews neutras
        return review
    else:
        review['text'] =  random.choice(negative_review)
        review['rating'] = random.randint(0, 4) # Notas baixas para reviews negativas
        return review

## 7. Configurar Kafka Producer

In [ ]:
# Configuração do produtor Kafka para enviar mensagens ao tópico desejado.
producer = KafkaProducer(
    bootstrap_servers=['kafka:9092'], # Endereço do broker Kafka
    value_serializer=lambda v: json.dumps(v).encode('utf-8'), # Serialização dos dados para JSON
    #acks='0'  # Não esperar confirmação do broker (opcional)
 )

# Verifica se a conexão com o broker foi estabelecida.
producer.bootstrap_connected()

## 8. Enviar mensagens fictícias a cada 10 segundos

In [ ]:
# Loop para enviar 1000 mensagens fictícias ao Kafka, uma a cada 10 segundos.
for _ in range(1000):
    # Seleciona um cliente e um filme aleatoriamente.
    customer = customers.sample(1).iloc[0]
    film = films.sample(1).iloc[0]
    # Gera uma review aleatória.
    review = generate_review()
    # Monta o payload da mensagem.
    payload = {
        'name': customer['name'],
        'film': film['title'],
        'rating': review['rating'],
        'review': review['text'],
    }
    # Envia a mensagem para o tópico 'popular_critics'.
    producer.send(
        topic='popular_critics',
        value=payload
    )
    # Aguarda 10 segundos antes de enviar a próxima mensagem.
    time.sleep(10)

## Limpando e encerrando canal

In [ ]:
# Garante que todas as mensagens foram enviadas e encerra o produtor Kafka.
producer.flush()
producer.close()